In [85]:
import pymysql
import pandas as pd

class conectarDB():
    def __init__(self,host,user,password,db):
        self.host = host
        self.user = user
        self.password = password
        self.db = db
        __cursor = ''
        self.__connect()

    def __connect(self):
        try:
            self.baseDatos = pymysql.connect(
                host = self.host, 
                user = self.user,
                password = self.password,
                db = self.db)

            self.__cursor = self.baseDatos.cursor()
            print("Conexión correcta")
            return True

        except (pymysql.err.OperationalError, pymysql.err.InternalError) as e:
            print("Ocurrió un error al conectar: ", e)
            return False
    
    def __comprobarConexion(self):
        if hasattr(self,'baseDatos'):
            print ('Conexion abierta')
            return self.__cursor
        else:
            print ('Intentando reconexion')
            return self.__connect()
        

    def consultar(self, consulta):
        if self.__comprobarConexion():
            ejecucion = self.__cursor.execute(consulta)
            if ejecucion > 1:
                return self.__cursor.fetchall()
            return self.__cursor.fetchone()

    def modificar(self, consulta):
        if self.consultar(consulta):
            self.__baseDatos.commit()
    
    def dataFrame(self, consulta):
        if self.__comprobarConexion():
            return pd.read_sql_query(consulta, self.baseDatos)

    def close(self):
        if hasattr(self,'baseDatos') and self.baseDatos.open:
            self.baseDatos.close()
            print("Desconectado")    
    

Creamos el objeto employees.
Traemos todo el contenido de los departamentos.

In [16]:
employees = conectarDB('localhost','root','','employees')
consulta = 'SELECT * FROM departments'
departamentos = employees.consultar(consulta)
print(departamentos)

Conexión correcta
Conexion abierta
(('d009', 'Customer Service'), ('d005', 'Development'), ('d002', 'Finance'), ('d003', 'Human Resources'), ('d001', 'Marketing'), ('d004', 'Production'), ('d006', 'Quality Management'), ('d008', 'Research'), ('d007', 'Sales'))


In [11]:
consulta = 'SELECT * FROM employees'
e = employees.consultar(consulta)
e_df = employees.dataFrame(consulta)
e_df.head() 

Conexion abierta
Conexion abierta


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [9]:
cantidadMujeres = employees.consultar('SELECT count(*) FROM employees where gender ="F"')
print(cantidadMujeres)

Conexion abierta
(120051,)


In [13]:
consulta_staff = 'SELECT e.*, t.title from employees e INNER JOIN titles t on (e.emp_no = t.emp_no) where t.title = "Staff"'
s = employees.consultar(consulta_staff)
s_df = employees.dataFrame(consulta_staff)
s_df.head() 

Conexion abierta
Conexion abierta


,emp_no,birth_date,first_name,last_name,gender,hire_date,title
0,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff
1,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,Staff
2,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10,Staff
3,10011,1953-11-07,Mary,Sluis,F,1990-01-22,Staff
4,10016,1961-05-02,Kazuhito,Cappelletti,M,1995-01-27,Staff
